# The Real code

In [2]:
using Random, Distributions, Plots

What needs to be done:
. Statistics along the way
. Rebalancing ?
. ArrivalProfiles
. Last simulation
... Plots
... Time to run consecutively (does that have something to do with rebalancing?)
... 

Questions for later:
. a total number of runs (a single run has the length of the number of days per run) ??
. Struct for e?
. The runtime, and rebalancing time
... Travel time, mister vi ikke biler?? Hvordan skal man gøre det
... Pretty much just rebalancing!
. what to do when travel time passes total time?

In [3]:
# Using tuples instead of PriorityQueue, because they can have multiple identical keys and the queue cannot.
# This is slightly slower but not by that much, not really a noticable difference in this case. 
# You have to get real up there in iterations before it really chugs behind.

# mutable struct Events
#     e::Array{Tuple{Float64,SubString},1}
# end
# Figure out if this can be a struct !!!

struct Environment
    hours::Float64
    lambda::Float64  #arrival rate
    mu::Float64 #service rate
   
   #Add arrival rate per hour for both stations. Add travel-time rate for both stations.
    Environment() = new(
    2, 
    (1/6), 
    (1/12)
    )
end

mutable struct Stats
    n_cars_st1::Int64
    n_cars_st2::Int64
    unfilled_st1::Int64
    unfilled_st2::Int64
    filled_st1::Int64
    filled_st2::Int64
    empty_st1::Float64
    empty_st2::Float64
end

# Might not be just as fast as PriorityQueue but it is not that much behind
function insert_and_sort!(vect::Vector, x)
    (splice!(vect, searchsorted(vect,x), [x]); vect) 
end

insert_and_sort! (generic function with 1 method)

In [19]:
function arrivals_generator(env::Environment)
    e = Array{Tuple{Float64,String},1}()
    # Arrivals at station 1
    push!(e,(rand(Exponential(env.lambda)),"st1")) # First arrival
    while last(e)[1] < env.hours
       push!(e,(rand(Exponential(env.lambda))+last(e)[1],"st1"))
    end
    # Fix den sidste den er over 2

    pop!(e)
    last_st1 = last(e)[1] # Storing last time (for later use to time being empty)

    # Arrivals at station 2 (repeat of station 1)
    push!(e,(rand(Exponential(env.lambda)),"st2"))
    while last(e)[1] < env.hours
        push!(e,(rand(Exponential(env.lambda))+last(e)[1],"st2"))
    end

    pop!(e)
    last_st2 = last(e)[1]

    # Sorting the arrivals so first time is 1 and so on
    sort!(e)

    return e, last_st1, last_st2
end

arrivals_generator (generic function with 1 method)

In [10]:
# Do this:
# stats.overview_usage = [stats.overview_usage; t state.queue_length state.server_busy]
# Real nice, steal that shit!

function arrival(station::String,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats)
    # WHAT to do when travel time passes total time?? makes sense? ask teacher
    if station == "st1"
        # Student arrives at empty station. That is one unsatisfied costumer :(
        if stat.n_cars_st1 == 0
            stat.unfilled_st1 += 1
            popfirst!(e)
        else
            stat.n_cars_st1 -= 1
            stat.filled_st1 += 1
            
            insert_and_sort!(e,(rand(Exponential(env.mu))+e[1][1],"que_st2")) 

            if stat.n_cars_st1 == 0 # station becomes empty
                stat.empty_st1 -= popfirst!(e)[1] # Empty station timer starts 
            else
                popfirst!(e)
            end
        end
        
    elseif station == "st2"
        if stat.n_cars_st2 == 0
            stat.unfilled_st2 += 1
            popfirst!(e)
        else
            stat.n_cars_st2 -= 1
            stat.filled_st2 += 1
            
            insert_and_sort!(e,(rand(Exponential(env.mu))+e[1][1],"que_st1") )

            if stat.n_cars_st2 == 0 # station becomes empty
                stat.empty_st2 -= popfirst!(e)[1] # Empty station timer starts 
            else
                popfirst!(e)
            end

        end

    else
        throw(ErrorException("Something went wrong in sim()."))
    end

end 

function travel(station::String,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats)

    if station == "que_st1" # If car arrives at st2
        if stat.n_cars_st1 == 0 # Arriving to an empty station
            stat.n_cars_st1 += 1
            stat.empty_st1 += popfirst!(e)[1] # End empty timer
        else
            stat.n_cars_st1 += 1
            popfirst!(e)    
        end
    elseif station == "que_st2" # If car arrives at st2
        if stat.n_cars_st2 == 0 # Arriving to an empty station
            stat.n_cars_st2 += 1 
            stat.empty_st2 += popfirst!(e)[1] # End empty timer
        else
            stat.n_cars_st2 += 1
            popfirst!(e)    
        end
    else 
        throw(ErrorException("Something went wrong in sim()."))
    end

end


travel (generic function with 1 method)

In [11]:
function sim(env::Environment,stat::Stats)
    e, last_st1, last_st2 = arrivals_generator(env)

    # Missing statistics all the way through
    while !isempty(e)
        load_type = e[1][2]

        if load_type == "st1" || load_type == "st2"
            arrival(load_type,e,env,stat)
        elseif load_type == "que_st1" || load_type == "que_st2"
            travel(load_type,e,env,stat)
        else
            throw(ErrorException("Issue in the event type!"))
        end
    end
    
    # Adding closing time if the station is still empty
    if stat.n_cars_st1 == 0
        stat.empty_st1 += last_st1
    elseif stat.n_cars_st2 == 0
        stat.empty_st2 += last_st2
    end

    return stat.n_cars_st1,stat.n_cars_st2,stat.unfilled_st1,stat.unfilled_st2,stat.filled_st1,stat.filled_st2,stat.empty_st1,stat.empty_st2
end

sim (generic function with 1 method)

### Testing

In [16]:
env = Environment()
stat = Stats(5,5,0,0,0,0,0,0)
sim(env,stat)

(9, 1, 0, 1, 8, 12, 0.0, 0.5633268233408981)

In [47]:
env = Environment()
e = Array{Tuple{Float64,String},1}()
arrivals_generator(e,env)
e[1][2]

"st1"

In [6]:
v = Array{Tuple{Float64,String},1}([(1,"1"),(4,"2"),(223,"3")])
sort!(v)
insert_and_sort!(v, (2,"4"))


4-element Vector{Tuple{Float64, String}}:
 (1.0, "1")
 (2.0, "4")
 (4.0, "2")
 (223.0, "3")

In [24]:
env = Environment()
arrivals_generator(env)

([(0.050215679619025025, "st1"), (0.11753000795924158, "st1"), (0.12099254259190709, "st2"), (0.1238659694301058, "st2"), (0.31774564922106496, "st2"), (0.4551077722700152, "st2"), (0.5399029527630247, "st2"), (0.6441339101761747, "st1"), (0.6808171007791013, "st1"), (0.7350848205986377, "st2")  …  (0.9405867328751559, "st1"), (1.2570578495574223, "st2"), (1.2993468276099027, "st1"), (1.450768400523547, "st2"), (1.5511538832014327, "st1"), (1.6302673831512882, "st1"), (1.688188517923613, "st2"), (1.725905532273659, "st1"), (1.9349099694450653, "st2"), (1.9832097148576344, "st2")], 1.725905532273659, 1.9832097148576344)